In [1]:
from PIL import ImageDraw, ImageFont, Image

def pdf_bbox_to_img_bbox(bbox_pdf, scale_x, scale_y):
    x0, y0, x1, y1 = bbox_pdf
    return (
        int(x0 / scale_x),
        int(y0 / scale_y),
        int(x1 / scale_x),
        int(y1 / scale_y),
    )

def visualize_page(page, viz_items, dpi=300, save_path=None):
    # ---- render page ----
    pix = page.get_pixmap(dpi=dpi)
    img = Image.frombytes("RGB", (pix.width, pix.height), pix.samples)

    draw = ImageDraw.Draw(img)

    try:
        font = ImageFont.truetype("arial.ttf", 16)
    except:
        font = ImageFont.load_default()

    for item in viz_items:
        scale_x = item["scale_x"]
        scale_y = item["scale_y"]

        # ---- figure bbox ----
        fig_bbox = pdf_bbox_to_img_bbox(
            item["figure_bbox_pdf"], scale_x, scale_y
        )
        fig_color = "red" if item["figure_type"] == "Table" else "blue"

        draw.rectangle(fig_bbox, outline=fig_color, width=3)
        draw.text(
            (fig_bbox[0], fig_bbox[1] - 18),
            item["figure_type"],
            fill=fig_color,
            font=font,
        )

        if item["caption_bbox_pdf"] is not None:
            # ---- caption bbox ----
            cap_bbox = pdf_bbox_to_img_bbox(
                item["caption_bbox_pdf"], scale_x, scale_y
            )
    
            draw.rectangle(cap_bbox, outline="green", width=3)
    
            caption_preview = item["caption_text"][:60].replace("\n", " ")
            draw.text(
                (cap_bbox[0], cap_bbox[3] + 2),
                caption_preview,
                fill="green",
                font=font,
            )

    if save_path:
        img.save(save_path)

    return img



In [3]:
from app.pdf_agent import PDFTableExtractor
import fitz

extractor = PDFTableExtractor(device="mps")

doc = fitz.open("data/downloads/manual_down/alphabet.pdf")

for i, page in enumerate(doc):
    outs, viz_items = extractor.extract(page)
    if not viz_items:
        continue

    img = visualize_page(
        page,
        viz_items,
        dpi=extractor.dpi,
        save_path=f"data/extracted_tables/page_{i:03d}_viz.png",
    )

Downloaded to data/models/yolov11x_best.pt


Some weights of the model checkpoint at microsoft/table-transformer-structure-recognition were not used when initializing TableTransformerForObjectDetection: ['model.backbone.conv_encoder.model.layer2.0.downsample.1.num_batches_tracked', 'model.backbone.conv_encoder.model.layer3.0.downsample.1.num_batches_tracked', 'model.backbone.conv_encoder.model.layer4.0.downsample.1.num_batches_tracked']
- This IS expected if you are initializing TableTransformerForObjectDetection from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TableTransformerForObjectDetection from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at Fa


0: 640x512 1 Section-header, 13 Texts, 1 Title, 383.8ms
Speed: 2.7ms preprocess, 383.8ms inference, 0.6ms postprocess per image at shape (1, 3, 640, 512)

0: 640x512 10 Texts, 428.5ms
Speed: 2.0ms preprocess, 428.5ms inference, 1.1ms postprocess per image at shape (1, 3, 640, 512)

0: 640x512 1 Page-header, 2 Section-headers, 1 Table, 545.4ms
Speed: 2.2ms preprocess, 545.4ms inference, 0.9ms postprocess per image at shape (1, 3, 640, 512)


Batches: 100%|██████████| 1/1 [00:00<00:00,  2.36it/s]



=========== TABLE CAPTION (MARKDOWN QUERY) ===========
QUERY:
Which sentence is the title or caption describing the following table?

| PART I |  |
| --- | --- |
| Item 1. | Business |
| Item 1A. | Risk Factors |
| Item 1B. | Unresolved Staff Comments |
| Item 1C. | Cybersecurity |
| Item 2. | Properties |
| Item 3. | Legal Proceedings |
| Item 4. | Mine Safety Disclosures |
| PART II |  |
| Item 5. | Market for Registrant’s Common Equity, Related Stockholder Matters, and Issuer Purchases of Equity Securities |
| Item 6. | [Reserved] |
| Item 7. | Management’s Discussion and Analysis of Financial Condition and Results of Operations |
| Item 7A. | Quantitative and Qualitative Disclosures About Market Risk |
| Item 8. | Financial Statements and Supplementary Data |
| Item 9. | Changes in and Disagreements With Accountants on Accounting and Financial Di 

Rank 1 | Score -8.4740
TABLE OF CONTENTS
------------------------------------------------------------
Rank 2 | Score -9.1990
Alphabet 

...


=========== TABLE CAPTION (MARKDOWN QUERY) ===========
QUERY:
Which sentence is the title or caption describing the following table?

| Exhibit Number 4.07 |  | Description Transfer Restriction Agreement, dated October 2, 2015, between the Registrant and Eric E. Schmidt and certain | Form Current Report on Form 8-K (File No. 001-37580) | Date October 2, 2015 |
| --- | --- | --- | --- | --- |
| 4.08 |  | of its affiliates Class C Undertaking, dated October 2, 2015, executed by the Registrant | Current Report on Form 8-K (File No. 001-37580) | October 2, 2015 |
| 4.09 |  | Indenture, dated February 12, 2016, between the Registrant and The Bank of New York Mellon Trust | Registration Statement on Form S-3 (File No. 333-209510) | February 12, 2016 |
| 4.10 |  | Company, N.A., as Trustee Registrant Registration Rights Agreement dated | Registration Statem 

Rank 1 | Score -6.9825
Table of Contents
Alphabet Inc.
------------------------------------------------------------
Rank 2 | Score -7.

Batches: 100%|██████████| 1/1 [00:00<00:00, 17.37it/s]



=========== TABLE CAPTION (MARKDOWN QUERY) ===========
QUERY:
Which sentence is the title or caption describing the following table?

| Number 10.08.2 | u | Description Alphabet Inc. Amended and Restated 2021 Stock Plan | Form Quarterly Report on Form 10-Q (File | Date July 26, 2023 |
| --- | --- | --- | --- | --- |
| 10.08.3 | u | - Form of Alphabet Restricted Stock Unit Agreement Alphabet Inc. Amended and Restated 2021 Stock Plan – Form of Alphabet Restricted Stock Unit Agreement | No. 001-37580) Quarterly Report on Form 10-Q (File No. 001-37580) | April 26, 2024 |
| 10.08.4 | u | Alphabet Inc. Amended and Restated 2021 Stock Plan - Form of Alphabet 2022 CEO Performance Stock Unit | Annual Report on Form 10-K (File No. 001-37580) | February 3, 2023 |
| 10.08.5 | u | Agreement Alphabet Inc. Amended and Restated 2021 Stock Plan – Form of Alphabet CEO 

Rank 1 | Score -6.7112
Table of Contents
Alphabet Inc.
------------------------------------------------------------
Rank 2 | Score -7.

Batches: 100%|██████████| 1/1 [00:00<00:00,  8.20it/s]



=========== TABLE CAPTION (MARKDOWN QUERY) ===========
QUERY:
Which sentence is the title or caption describing the following table?

| /S/ SUNDAR PICHAI | /S/ SUNDAR PICHAI | Chief Executive Officer and Director (Principal Executive Officer) | February 4, 2025 |
| --- | --- | --- | --- |
|  | Sundar Pichai | Senior Vice President and Chief Financial Officer |  |
|  | /S/ ANAT ASHKENAZI Anat Ashkenazi | (Principal Financial Officer) | February 4, 2025 |
|  |  | Vice President, Corporate Controller and Principal |  |
| /S/ | AMIE THUENER O'TOOLE | Accounting Officer | February 4, 2025 |
| /S/ | Amie Thuener O'Toole |  |  |
| /S/ | FRANCES H. ARNOLD | Director | February 4, 2025 |
| /S/ | Frances H. Arnold |  |  |
| /S/ | /S/ SERGEY BRIN | Co-Founder and Director | February 4, 2025 |
|  | Sergey Brin |  |  |
| /S/ | R. MARTIN CHAVEZ | Director | Februa 

Rank 1 | Score -8.1283
Table of Contents
Alphabet Inc.
------------------------------------------------------------
Rank 2 | Score -8.

In [52]:
import torch
from transformers import AutoTokenizer, RobertaForSequenceClassification

tokenizer = AutoTokenizer.from_pretrained("FacebookAI/roberta-base")
model = RobertaForSequenceClassification.from_pretrained("FacebookAI/roberta-base")

# To train a model on `num_labels` classes, you can pass `num_labels=num_labels` to `.from_pretrained(...)`
model = RobertaForSequenceClassification.from_pretrained("FacebookAI/roberta-base", num_labels=2)
# model.classifier.load_state_dict(torch.load("data/roberta_figure_diff.pt"))

for name, param in model.named_parameters():
    if "classifier" not in name:
        param.requires_grad_(False)

Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at FacebookAI/roberta-base and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at FacebookAI/roberta-base and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [53]:
import torch
from torch.utils.data import random_split, Dataset, DataLoader
import pandas as pd

class ClassifDataset(Dataset):
    def __init__(self):
        super().__init__()
        self.data = pd.read_csv("dataset_example.csv")

    def __getitem__(self, index):
        return self.data.loc[index]["text"], self.data.loc[index]["label"]
    
    def __len__(self):
        return len(self.data)

torch.manual_seed(42)
device = "cuda" if torch.cuda.is_available() else "mps" if torch.mps.is_available() else "cpu"

dataset = ClassifDataset()
train_data, test_data = random_split(dataset, [0.8, 0.2])

train = DataLoader(train_data, batch_size=32, shuffle=True)
test = DataLoader(test_data, batch_size=32, shuffle=False)

print(train, test)

for X, y in train:
    print(X, y)
    break

optim = torch.optim.AdamW(model.parameters(), lr=5e-4)

<torch.utils.data.dataloader.DataLoader object at 0x3d06ea610> <torch.utils.data.dataloader.DataLoader object at 0x3d06e9810>
('The following table presents OI&E, (in millions)', 'Wage gains have also moderated as economic momentum weakens.', 'Investment in intellectual property has become increasingly important for long-term competitiveness.', 'After revising his business plan, he secured financing to open a new clinic in downtown Toronto.', 'Despite advances in medical imaging and minimally invasive treatments, access to specialized care remains uneven. Patients in smaller communities often travel long distances for procedures that are readily available in major cities. Policymakers have identified this disparity as a growing concern.', 'Automotive R&D investment by region', 'The table beside shows the revenue of our company from time to time', 'AGRICULTURAL OUTPUT BY CROP', 'While inflation has eased, price pressures remain elevated in some sectors.', 'Table 5. Hyperparameter settin

In [54]:
import torch.nn.functional as F


model.to(device)
epochs = 90

ce = torch.nn.CrossEntropyLoss()

for i in range(epochs):
    model.train()

    epoch_loss = 0
    epoch_test_loss = 0
    for X, y in train:
        tokens = tokenizer(X, return_tensors="pt", padding=True).to(device)
        y = y.to(device)
        ypred = model(**tokens).logits
        
        loss = ce(ypred, y)

        optim.zero_grad()
        loss.backward()
        optim.step()

        epoch_loss += loss.item()

    model.eval()
    correct = 0
    total_dt = 0
    with torch.no_grad():
        for X_test, y_test in test:
            tokens = tokenizer(X_test, return_tensors="pt", padding=True).to(device)
            y_test = y_test.to(device)
            ypred_test = model(**tokens).logits

            loss = ce(ypred_test, y_test)
            epoch_test_loss += loss.item()
            correct += torch.sum((ypred_test.argmax(dim=1) == y_test).int())
            total_dt += len(X_test)

    print(f"loss = {epoch_loss / len(train)}", f"test loss = {epoch_test_loss / len(test)} accuracy = {correct / total_dt}")


loss = 0.7058371802171072 test loss = 0.7293407320976257 accuracy = 0.4146341383457184
loss = 0.7043639322121938 test loss = 0.6510231494903564 accuracy = 0.6829268336296082
loss = 0.6614579657713572 test loss = 0.6217287182807922 accuracy = 0.5609756112098694
loss = 0.6533557971318563 test loss = 0.6566230356693268 accuracy = 0.5121951103210449
loss = 0.6139521400133768 test loss = 0.6204953193664551 accuracy = 0.7804877758026123
loss = 0.5965128938357035 test loss = 0.5932426154613495 accuracy = 0.7804877758026123
loss = 0.5825155476729075 test loss = 0.5595134198665619 accuracy = 0.6585366129875183
loss = 0.5465270976225535 test loss = 0.5432136654853821 accuracy = 0.6829268336296082
loss = 0.5650758743286133 test loss = 0.5584767460823059 accuracy = 0.7560975551605225
loss = 0.505209356546402 test loss = 0.5251592993736267 accuracy = 0.7804877758026123
loss = 0.5170064568519592 test loss = 0.4896533638238907 accuracy = 0.6829268336296082
loss = 0.5267351021369299 test loss = 0.4732

In [55]:
model.to(device)
model.eval()
tokens = tokenizer([
    "wi = 1"
], return_tensors="pt", padding=True).to(device)

logits = model(**tokens).logits
torch.softmax(logits, dim=-1)

tensor([[0.8222, 0.1778]], device='mps:0', grad_fn=<SoftmaxBackward0>)

In [56]:
torch.save(model.classifier.state_dict(), "data/roberta_figure_diff.pt")

In [15]:
import torch
from transformers import AutoTokenizer, RobertaForSequenceClassification

tokenizer = AutoTokenizer.from_pretrained("FacebookAI/roberta-base")
model = RobertaForSequenceClassification.from_pretrained("FacebookAI/roberta-base")

# inputs = tokenizer("Hello, my dog is cute", return_tensors="pt")

# with torch.no_grad():
#     logits = model(**inputs).logits

# predicted_class_id = logits.argmax().item()
# model.config.id2label[predicted_class_id]

# To train a model on `num_labels` classes, you can pass `num_labels=num_labels` to `.from_pretrained(...)`
model = RobertaForSequenceClassification.from_pretrained("FacebookAI/roberta-base", num_labels=2)
model.classifier.load_state_dict(torch.load("data/roberta_figure_diff.pt"))



Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at FacebookAI/roberta-base and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at FacebookAI/roberta-base and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


<All keys matched successfully>

In [32]:
model.to(device)
model.eval()
tokens = tokenizer([
    "Financial Asset Loss per Annum"
], return_tensors="pt", padding=True).to(device)

logits = model(**tokens).logits
torch.softmax(logits, dim=-1)

tensor([[0.1464, 0.8536]], device='mps:0', grad_fn=<SoftmaxBackward0>)